In [7]:
#import os
#os.chdir('../')

In [1]:
# Modify the sys.path
import sys
sys.path.append(r"E:\Code\eptic\src")

# Import the necessary functions
from db_connect import get_database_config_from_json, connect_to_db

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import os

import hydra
from omegaconf import DictConfig

import mariadb
import json

Notebook to figure out what's wrong with the "duration" field of some languages

In [24]:
hydra.core.global_hydra.GlobalHydra.instance().clear()

In [2]:
from hydra import initialize, compose

# Use the 'initialize' and 'compose' functions for Hydra
config_dir_path = r"E:\Code\eptic\config"

with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name='main.yaml')  # Adjust this if your filename differs

config_data = get_database_config_from_json(cfg.db.credentials)
print(f"Connecting to database: {config_data['name']}")
conn = connect_to_db(config_data)

# Function to get all table names in the database
def get_table_names(connection):
    return pd.read_sql_query("SHOW TABLES;", connection)

# Function to query a table and return a dataframe with prefixed column names
def get_table_data(table_name, connection):
    query = f"SELECT * FROM `{table_name}`;"
    df = pd.read_sql_query(query, connection)
    df.columns = [f"{table_name}.{col}" for col in df.columns]
    return df

# Get a list of all table names
table_names_df = get_table_names(conn)
table_names = table_names_df[table_names_df.columns[0]].tolist()  # Adjust the index if necessary

# Iterate through table names, get their data, and optionally concatenate into a single DataFrame
all_tables_data = pd.DataFrame()  # Initialize an empty DataFrame to hold all data
for table_name in table_names:
    table_data = get_table_data(table_name, conn)
    # Here you can either export each table to a CSV or combine them
    # To export to CSV:
    table_data.to_csv(f"{table_name}_data.csv", index=False)
    # Or to concatenate them into a single DataFrame (assuming they can be concatenated):
    # all_tables_data = pd.concat([all_tables_data, table_data], axis=0)

# If you have concatenated all tables and want to save the entire database to a single CSV:
# all_tables_data.to_csv("all_tables_data.csv", index=False)

Connecting to database: eptic


,lang
0,pl
1,pl
2,fr
3,fr
4,en
...,...
976,fi
977,fi
978,fi
979,fi
